In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
import numpy as np
import re

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dossier='/content/drive/MyDrive/'
df=pd.read_csv(dossier+'text_preprocessed_boubacar.csv')
df=df.astype(str).sample(df.shape[0]//150)
df_questions=df["Title"]
df_answers=df["Answers"]

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.head()


,Unnamed: 0,Title,Content,Answers
36116,36116,pythonic way open file given one argument stdin ?,trying write python script follows common unix...,known problem xml parsers often send http requ...
22152,22152,run function every x minutes - python,using python pygtk . interested running certai...,linux server : 2.6.18 rhel5 gcc version : gcc ...
83760,83760,open ssh tunnel using python ?,trying connect remote mysql database using dja...,nan
300640,300640,long dataframe apply row functions,read csv file time value columns get dataframe...,nan
129219,129219,path variable python,change environmental path variable code python...,"would like use aquamacs edit python files , ch..."


In [5]:
questions_train, questions_test, answers_train, answers_test = train_test_split(df_questions,df_answers, test_size=0.2, random_state=42)

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential

# Créer un tokenizer pour convertir les phrases en séquences de mots
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions_train)

# Convertir les phrases en séquences de mots
questions_train = tokenizer.texts_to_sequences(questions_train)
questions_test = tokenizer.texts_to_sequences(questions_test)


# Créer un tokenizer pour convertir les phrases en séquences de mots
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(answers_train)

# Convertir les phrases en séquences de mots
answers_train = tokenizer.texts_to_sequences(answers_train)
answers_test = tokenizer.texts_to_sequences(answers_test)

max_length_questions = max([len(s) for s in questions_train])
max_length_answers = max([len(s) for s in answers_train])
max_length=max([max_length_questions, max_length_answers])
# Pad les séquences pour qu'elles aient toutes la même longueur

questions_train = tf.keras.preprocessing.sequence.pad_sequences(questions_train, maxlen=max_length)
questions_test = tf.keras.preprocessing.sequence.pad_sequences(questions_test, maxlen=max_length)


# Pad les séquences pour qu'elles aient toutes la même longueur

answers_train = tf.keras.preprocessing.sequence.pad_sequences(answers_train, maxlen=max_length)
answers_test = tf.keras.preprocessing.sequence.pad_sequences(answers_test, maxlen=max_length)

# Créer un encoder-decoder avec un LSTM
encoder_inputs = Input(shape=(max_length,1))
encoder = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length_questions)(encoder_inputs)
encoder = LSTM(64, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_length,))
decoder_embedding = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length)(decoder_inputs)
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(tokenizer.word_index)+1, activation='softmax')
outputs = decoder_dense(decoder_outputs)


In [7]:

# Compiler le modèle
model = Model([encoder_inputs, decoder_inputs], [outputs])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Entraîner le modèle
model.fit([questions_train, questions_train], answers_train, epochs=10, batch_size=8, validation_data=([questions_test, answers_test], answers_test), steps_per_epoch=2, validation_steps=2)


Epoch 1/10
2/2 [==============================] - 7s 2s/step - loss: 9.7009 - accuracy: 9.5566e-05 - val_loss: 9.6912 - val_accuracy: 4.7783e-05
Epoch 2/10
2/2 [==============================] - 1s 743ms/step - loss: 9.6878 - accuracy: 0.4927 - val_loss: 9.6781 - val_accuracy: 0.9670
Epoch 3/10
2/2 [==============================] - 1s 770ms/step - loss: 9.6742 - accuracy: 0.9772 - val_loss: 9.6621 - val_accuracy: 0.9694
Epoch 4/10
2/2 [==============================] - 1s 754ms/step - loss: 9.6567 - accuracy: 0.9734 - val_loss: 9.6379 - val_accuracy: 0.9701
Epoch 5/10
2/2 [==============================] - 1s 766ms/step - loss: 9.6278 - accuracy: 0.9861 - val_loss: 9.5951 - val_accuracy: 0.9706
Epoch 6/10
2/2 [==============================] - 1s 1s/step - loss: 9.5790 - accuracy: 0.9524 - val_loss: 9.5098 - val_accuracy: 0.9709
Epoch 7/10
2/2 [==============================] - 1s 1s/step - loss: 9.4704 - accuracy: 0.9863 - val_loss: 9.3442 - val_accuracy: 0.9710
Epoch 8/10
2/2 [=====

In [11]:
# Évaluer le modèle sur les données de test
loss, acc = model.evaluate([questions_test, questions_test], answers_test, verbose=0, batch_size=8)
print("Test accuracy:", acc)

Test accuracy: 0.9744859933853149


In [8]:

pred = tokenizer.sequences_to_matrix(text, mode='binary')
model.predict(pred)

In [9]:

#tableau=np.array([questions_train, answers_train])
#tableau.reshape(1,tableau.shape[1],tableau.shape[2])